In [96]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.output_parsers import JsonOutputParser
from datetime import datetime
from langchain.output_parsers import DatetimeOutputParser
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
chat = ChatOpenAI()
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [97]:
class HistoryQuiz():

    def create_history_question(self,topic):
        system_template = "You are giving a history quiz. I want you to generate a random single history question for me about {topic}. This historical question needs to have a date as the correct answer. Only return the quiz question."
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

        human_template = "{question_request}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

        q = "Give me a quiz question where the correct answer is a specific date."
        request = chat_prompt.format_prompt(topic=topic,question_request=q).to_messages()

        result = llm.invoke(request)
        return result.content

    def get_AI_answer(self,question):

    #datetime parse
       output_parser = DatetimeOutputParser()
       json_parser = JsonOutputParser()

    #syst template
       system_template = "You answer quiz questions with just a date and parse the date"
       system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

    #human template -- format instructions
       human_template = """Answer the user's question:
       {question}
       {format_instructions}
       """
       human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

    #compile ChatPromptTemplate
       chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

    # insert question and get_format_instructions
       request = chat_prompt.format_prompt(question=question, format_instructions=output_parser.get_format_instructions()).to_messages()

    #chatbot result -- format to datetime
       result = llm.invoke(request)
       correct_datetime = output_parser.parse(result.content)
       return correct_datetime

    def get_user_answer(self,question):

        print(question)
        print("\n")

        year = int(input("Enter the year: "))
        month = int(input("Enter the month (1-12): "))
        day = int(input("Enter the day (1-31): "))
        user_datetime = datetime(year, month, day)

        return user_datetime

    def check_user_answer(self,user_answer,ai_answer):

        # print or return the difference between the answers here!
        difference = user_answer - ai_answer
        formatted_difference = str(difference)
        print("The difference between the answer and your guess is: ", formatted_difference)
        pass

In [98]:
quiz_bot = HistoryQuiz()

In [99]:
question = quiz_bot.create_history_question(topic='Rome')
question

'What year did the Roman Empire officially fall, marking the end of ancient Rome’s dominance in the West?'

In [100]:
ai_answer = quiz_bot.get_AI_answer(question)

In [101]:
ai_answer

datetime.datetime(480, 9, 4, 0, 0)

In [102]:
user_answer = quiz_bot.get_user_answer(question)

What year did the Roman Empire officially fall, marking the end of ancient Rome’s dominance in the West?




KeyboardInterrupt: Interrupted by user

In [103]:
user_answer

datetime.datetime(466, 6, 6, 0, 0)

In [104]:
quiz_bot.check_user_answer(user_answer,ai_answer)

The difference between the answer and your guess is:  -5204 days, 0:00:00
